# 🔢 Vector & Matrix Operations Tutorial

이 노트북에서는 NumPy를 활용한 벡터/행렬 연산을 실습합니다.
스칼라 연산에서 벡터 연산으로 전환하여 효율성을 극대화합니다.

## 1. 환경 설정

In [1]:
import numpy as np
import time
import sys
import os

# 상위 디렉토리를 path에 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('.')))))

# core 모듈 import
from core.tensor_ops import (
    matmul, relu, sigmoid, softmax,
    mse_loss, cross_entropy, batch_norm,
    one_hot, accuracy
)
from core.nn_vectorized import LinearLayer, MLPVectorized, SGDOptimizer

print("✅ 환경 설정 완료!")
print(f"NumPy 버전: {np.__version__}")

✅ 환경 설정 완료!
NumPy 버전: 2.3.1


## 2. NumPy 기초: 스칼라 vs 벡터

In [2]:
# 스칼라 연산 (느림)
def scalar_add(a, b):
    result = []
    for i in range(len(a)):
        result.append(a[i] + b[i])
    return result

# 벡터 연산 (빠름)
def vector_add(a, b):
    return a + b

# 성능 비교
size = 100000
a_list = list(range(size))
b_list = list(range(size))
a_array = np.array(a_list)
b_array = np.array(b_list)

# 스칼라 연산 시간
start = time.time()
result_scalar = scalar_add(a_list, b_list)
scalar_time = time.time() - start

# 벡터 연산 시간
start = time.time()
result_vector = vector_add(a_array, b_array)
vector_time = time.time() - start

print(f"🐌 스칼라 연산: {scalar_time:.4f}초")
print(f"🚀 벡터 연산: {vector_time:.4f}초")
print(f"⚡ 속도 향상: {scalar_time/vector_time:.1f}배 빠름!")

🐌 스칼라 연산: 0.0042초
🚀 벡터 연산: 0.0002초
⚡ 속도 향상: 19.3배 빠름!


## 3. Broadcasting 이해하기

In [3]:
# Broadcasting 예제 1: 스칼라와 벡터
print("📍 예제 1: 스칼라 + 벡터")
a = np.array([1, 2, 3])
b = 10
result = a + b
print(f"  {a} + {b} = {result}")
print()

# Broadcasting 예제 2: 벡터와 행렬
print("📍 예제 2: 행렬 + 벡터 (행 방향)")
matrix = np.array([[1, 2, 3],
                   [4, 5, 6]])
vector = np.array([10, 20, 30])
result = matrix + vector
print(f"  행렬:\n{matrix}")
print(f"  벡터: {vector}")
print(f"  결과:\n{result}")
print()

# Broadcasting 예제 3: 열 방향
print("📍 예제 3: 행렬 + 벡터 (열 방향)")
vector_col = np.array([[10], [20]])
result = matrix + vector_col
print(f"  행렬:\n{matrix}")
print(f"  벡터:\n{vector_col}")
print(f"  결과:\n{result}")

📍 예제 1: 스칼라 + 벡터
  [1 2 3] + 10 = [11 12 13]

📍 예제 2: 행렬 + 벡터 (행 방향)
  행렬:
[[1 2 3]
 [4 5 6]]
  벡터: [10 20 30]
  결과:
[[11 22 33]
 [14 25 36]]

📍 예제 3: 행렬 + 벡터 (열 방향)
  행렬:
[[1 2 3]
 [4 5 6]]
  벡터:
[[10]
 [20]]
  결과:
[[11 12 13]
 [24 25 26]]


In [4]:
# Broadcasting 시각화
def visualize_broadcasting(a_shape, b_shape):
    """Broadcasting 과정 시각화"""
    print(f"Shape A: {a_shape}")
    print(f"Shape B: {b_shape}")
    
    # 차원 맞추기
    ndim = max(len(a_shape), len(b_shape))
    a_shape = (1,) * (ndim - len(a_shape)) + a_shape
    b_shape = (1,) * (ndim - len(b_shape)) + b_shape
    
    print(f"\n정렬 후:")
    print(f"  A: {a_shape}")
    print(f"  B: {b_shape}")
    
    # Broadcasting 가능 여부 확인
    result_shape = []
    for a, b in zip(a_shape, b_shape):
        if a == 1:
            result_shape.append(b)
        elif b == 1:
            result_shape.append(a)
        elif a == b:
            result_shape.append(a)
        else:
            print(f"\n❌ Broadcasting 불가능: {a} != {b}")
            return
    
    print(f"\n✅ 결과 Shape: {tuple(result_shape)}")

# 테스트
print("🔍 Broadcasting 분석")
print("=" * 30)
visualize_broadcasting((2, 3), (3,))
print("\n" + "=" * 30)
visualize_broadcasting((2, 1), (1, 3))
print("\n" + "=" * 30)
visualize_broadcasting((2, 3), (4,))  # 실패 케이스

🔍 Broadcasting 분석
Shape A: (2, 3)
Shape B: (3,)

정렬 후:
  A: (2, 3)
  B: (1, 3)

✅ 결과 Shape: (2, 3)

Shape A: (2, 1)
Shape B: (1, 3)

정렬 후:
  A: (2, 1)
  B: (1, 3)

✅ 결과 Shape: (2, 3)

Shape A: (2, 3)
Shape B: (4,)

정렬 후:
  A: (2, 3)
  B: (1, 4)

❌ Broadcasting 불가능: 3 != 4


## 4. 활성화 함수 벡터화

In [ ]:
# 활성화 함수 비교
x = np.linspace(-5, 5, 100)


# 각 활성화 함수 적용
y_relu = relu(x)
y_sigmoid = sigmoid(x)
y_tanh = np.tanh(x)

# 텍스트 그래프로 시각화
def plot_text_graph(y, title):
    """간단한 텍스트 그래프"""
    print(f"\n📊 {title}")
    print("  1.0 ┤")
    
    height = 10
    y_min, y_max = y.min(), y.max()
    
    for h in range(height, -1, -1):
        threshold = y_min + (y_max - y_min) * h / height
        line = "      │"
        
        for val in y[::5]:  # 샘플링
            if val >= threshold:
                line += "█"
            else:
                line += " "
        
        if h == height:
            print(f"  {y_max:3.1f} ┤" + line[6:])
        elif h == 0:
            print(f"  {y_min:3.1f} ┤" + line[6:])
        elif h == height // 2:
            print(f"  0.0 ┤" + line[6:])
        else:
            print(line)
    
    print("      └" + "─" * 20)
    print("       -5" + " " * 14 + "5")

plot_text_graph(y_relu, "ReLU")
plot_text_graph(y_sigmoid, "Sigmoid")
plot_text_graph(y_tanh, "Tanh")

[-5.         -4.8989899  -4.7979798  -4.6969697  -4.5959596  -4.49494949
 -4.39393939 -4.29292929 -4.19191919 -4.09090909 -3.98989899 -3.88888889
 -3.78787879 -3.68686869 -3.58585859 -3.48484848 -3.38383838 -3.28282828
 -3.18181818 -3.08080808 -2.97979798 -2.87878788 -2.77777778 -2.67676768
 -2.57575758 -2.47474747 -2.37373737 -2.27272727 -2.17171717 -2.07070707
 -1.96969697 -1.86868687 -1.76767677 -1.66666667 -1.56565657 -1.46464646
 -1.36363636 -1.26262626 -1.16161616 -1.06060606 -0.95959596 -0.85858586
 -0.75757576 -0.65656566 -0.55555556 -0.45454545 -0.35353535 -0.25252525
 -0.15151515 -0.05050505  0.05050505  0.15151515  0.25252525  0.35353535
  0.45454545  0.55555556  0.65656566  0.75757576  0.85858586  0.95959596
  1.06060606  1.16161616  1.26262626  1.36363636  1.46464646  1.56565657
  1.66666667  1.76767677  1.86868687  1.96969697  2.07070707  2.17171717
  2.27272727  2.37373737  2.47474747  2.57575758  2.67676768  2.77777778
  2.87878788  2.97979798  3.08080808  3.18181818  3

## 5. Softmax 구현 및 테스트

In [8]:
# Softmax 테스트
print("🎯 Softmax 함수 테스트")
print("=" * 40)

# 간단한 예제
logits = np.array([[1, 2, 3],
                   [1, 2, 3],
                   [3, 2, 1]])

probs = softmax(logits)

print("입력 (logits):")
print(logits)
print("\n출력 (probabilities):")
print(np.round(probs, 3))
print("\n각 행의 합:")
print(probs.sum(axis=1))

# 수치 안정성 테스트
print("\n🔒 수치 안정성 테스트")
large_logits = np.array([[1000, 1001, 1002]])
stable_probs = softmax(large_logits)
print(f"큰 값 입력: {large_logits[0]}")
print(f"안정적인 출력: {stable_probs[0]}")
print(f"합: {stable_probs.sum()}")
print(f"NaN 체크: {np.any(np.isnan(stable_probs))}")

🎯 Softmax 함수 테스트
입력 (logits):
[[1 2 3]
 [1 2 3]
 [3 2 1]]

출력 (probabilities):
[[0.09  0.245 0.665]
 [0.09  0.245 0.665]
 [0.665 0.245 0.09 ]]

각 행의 합:
[1. 1. 1.]

🔒 수치 안정성 테스트
큰 값 입력: [1000 1001 1002]
안정적인 출력: [0.09003057 0.24472847 0.66524096]
합: 0.9999999999999999
NaN 체크: False


## 6. 배치 처리 구현

In [22]:
class SimpleDataLoader:
    """간단한 DataLoader 구현"""
    
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_samples = len(X)
    
    def __iter__(self):
        indices = np.arange(self.n_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start_idx in range(0, self.n_samples, self.batch_size):
            batch_indices = indices[start_idx:start_idx + self.batch_size]
            # print(batch_indices)
            # print(self.X[batch_indices])
            # print(self.y[batch_indices])
            # print("\n")
            yield self.X[batch_indices], self.y[batch_indices]
    
    def __len__(self):
        return (self.n_samples + self.batch_size - 1) // self.batch_size

# 테스트
print("📦 DataLoader 테스트")
print("=" * 40)

# 가상 데이터
X = np.random.randn(100, 10)
y = np.random.randint(0, 3, 100)
print(X.shape)
print(y.shape)
print("len(X):", len(X))
print("X[0]:", X[0])
print("len(X[0]):", len(X[0]))
print("\n")

# DataLoader 생성
dataloader = SimpleDataLoader(X, y, batch_size=16)

print(f"총 샘플 수: {len(X)}")
print(f"배치 크기: 16")
print(f"총 배치 수: {len(dataloader)}")
print()

# 첫 3개 배치 확인
for i, (X_batch, y_batch) in enumerate(dataloader):
    if i >= 3:
        break
    print(f"배치 {i+1}: X shape={X_batch.shape}, y shape={y_batch.shape}")

📦 DataLoader 테스트
(100, 10)
(100,)
len(X): 100
X[0]: [-0.75479854 -0.08122968  1.14009243  0.32005746  0.56644043  0.271125
  1.6494374   1.7401719  -0.58618837 -0.23527365]
len(X[0]): 10


총 샘플 수: 100
배치 크기: 16
총 배치 수: 7

배치 1: X shape=(16, 10), y shape=(16,)
배치 2: X shape=(16, 10), y shape=(16,)
배치 3: X shape=(16, 10), y shape=(16,)


## 7. 벡터화된 신경망 구현

In [30]:
# 벡터화된 Linear Layer 테스트
print("🔗 Linear Layer 테스트")
print("=" * 40)

# Layer 생성
layer = LinearLayer(input_dim=10, output_dim=5, activation='relu')

# 배치 입력
X = np.random.randn(32, 10)  # 32개 샘플, 10차원

# Forward pass
output = layer.forward(X)
print(f"입력 shape: {X.shape}")
print(f"출력 shape: {output.shape}")
print(f"가중치 shape: {layer.W.shape}")
print(f"편향 shape: {layer.b.shape}")

# Backward pass
grad_output = np.random.randn(*output.shape)
grad_input = layer.backward(grad_output)
print(f"\n역전파:")
print(f"출력 그래디언트 shape: {grad_output.shape}")
print(f"입력 그래디언트 shape: {grad_input.shape}")
print(f"가중치 그래디언트 shape: {layer.dW.shape}")
print(f"편향 그래디언트 shape: {layer.db.shape}")

🔗 Linear Layer 테스트
입력 shape: (32, 10)
출력 shape: (32, 5)
가중치 shape: (10, 5)
편향 shape: (5,)

역전파:
출력 그래디언트 shape: (32, 5)
입력 그래디언트 shape: (32, 10)
가중치 그래디언트 shape: (10, 5)
편향 그래디언트 shape: (5,)


In [31]:
# 전체 MLP 테스트
print("🏗️ MLP 테스트")
print("=" * 40)

# MLP 생성
mlp = MLPVectorized(
    input_dim=10,
    hidden_dims=[20, 15],
    output_dim=3
)

print(f"모델 구조: 10 → 20 → 15 → 3")
print(f"레이어 수: {len(mlp.layers)}")

# 배치 처리
X = np.random.randn(32, 10)
output = mlp.forward(X)
print(f"\n입력 shape: {X.shape}")
print(f"출력 shape: {output.shape}")

# Softmax 적용
probs = mlp.predict_proba(X)
print(f"\n확률 분포 shape: {probs.shape}")
print(f"확률 합 (처음 5개): {probs.sum(axis=1)[:5]}")

🏗️ MLP 테스트
모델 구조: 10 → 20 → 15 → 3
레이어 수: 3

입력 shape: (32, 10)
출력 shape: (32, 3)

확률 분포 shape: (32, 3)
확률 합 (처음 5개): [1. 1. 1. 1. 1.]


## 8. XOR 문제 해결 (벡터화)

In [44]:
# XOR 데이터
X_xor = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])

y_xor = np.array([0, 1, 1, 0])

print("🎯 XOR 문제 (벡터화된 신경망)")
print("=" * 40)
print("데이터:")
for inputs, target in zip(X_xor, y_xor):
    print(f"  {inputs} → {target}")

# 모델 생성
xor_model = MLPVectorized(
    input_dim=2,
    hidden_dims=[4],
    output_dim=2  # 이진 분류를 2-class로
)

# 옵티마이저
optimizer = SGDOptimizer(xor_model, learning_rate=0.5)

# 학습
print("\n📈 학습 시작...")
for epoch in range(1000):
    # Forward
    logits = xor_model.forward(X_xor)
    probs = softmax(logits)
    
    # Loss
    loss = cross_entropy(probs, y_xor)
    
    # Backward
    grad = probs.copy()
    grad[np.arange(4), y_xor] -= 1
    grad /= 4
    
    xor_model.backward(grad)
    optimizer.step()
    
    if epoch % 200 == 0:
        acc = accuracy(probs, y_xor)
        print(f"Epoch {epoch:4d}: Loss={loss:.4f}, Acc={acc:.2f}")

# 최종 평가
print("\n📊 최종 결과:")
final_probs = xor_model.predict_proba(X_xor)
predictions = np.argmax(final_probs, axis=1)

for inputs, pred, target in zip(X_xor, predictions, y_xor):
    symbol = "✓" if pred == target else "✗"
    print(f"  {inputs} → 예측: {pred}, 정답: {target} {symbol}")

final_acc = accuracy(predictions, y_xor)
print(f"\n최종 정확도: {final_acc:.0%}")

🎯 XOR 문제 (벡터화된 신경망)
데이터:
  [0 0] → 0
  [0 1] → 1
  [1 0] → 1
  [1 1] → 0

📈 학습 시작...
Epoch    0: Loss=0.7683, Acc=0.75
Epoch  200: Loss=0.4542, Acc=0.75
Epoch  400: Loss=0.0665, Acc=1.00
Epoch  600: Loss=0.0269, Acc=1.00
Epoch  800: Loss=0.0162, Acc=1.00

📊 최종 결과:
  [0 0] → 예측: 0, 정답: 0 ✓
  [0 1] → 예측: 1, 정답: 1 ✓
  [1 0] → 예측: 1, 정답: 1 ✓
  [1 1] → 예측: 0, 정답: 0 ✓

최종 정확도: 100%


In [38]:
import numpy as np

grad = np.array([[0.7, 0.2, 0.1],
                 [0.1, 0.3, 0.6],
                 [0.2, 0.5, 0.3],
                 [0.8, 0.1, 0.1]])

y_xor = np.array([0, 2, 1, 0])

grad[np.arange(4), y_xor] -= 1
print(grad)
print(grad/4)

[[-0.3  0.2  0.1]
 [ 0.1  0.3 -0.4]
 [ 0.2 -0.5  0.3]
 [-0.2  0.1  0.1]]
[[-0.075  0.05   0.025]
 [ 0.025  0.075 -0.1  ]
 [ 0.05  -0.125  0.075]
 [-0.05   0.025  0.025]]


## 9. 성능 벤치마크

In [46]:
def benchmark_operations():
    """다양한 연산의 성능 비교"""
    
    print("⚡ 연산 성능 벤치마크")
    print("=" * 50)
    
    sizes = [100, 1000, 10000]
    
    for size in sizes:
        print(f"\n📏 크기: {size}")
        
        # 데이터 준비
        a = np.random.randn(size)
        b = np.random.randn(size)
        
        # 1. 덧셈
        start = time.time()
        for _ in range(1000):
            _ = a + b
        vector_time = time.time() - start
        
        start = time.time()
        for _ in range(1000):
            _ = [a[i] + b[i] for i in range(size)]
        scalar_time = time.time() - start
        
        print(f"  덧셈 - 벡터: {vector_time:.4f}s, 스칼라: {scalar_time:.4f}s, 속도향상: {scalar_time/vector_time:.1f}x")
        
        # 2. 행렬곱
        if size <= 1000:  # 큰 크기는 시간이 너무 오래 걸림
            A = np.random.randn(size, size)
            B = np.random.randn(size, size)
            
            start = time.time()
            _ = A @ B
            vector_time = time.time() - start
            
            print(f"  행렬곱 - NumPy: {vector_time:.4f}s")

benchmark_operations()

⚡ 연산 성능 벤치마크

📏 크기: 100
  덧셈 - 벡터: 0.0004s, 스칼라: 0.0135s, 속도향상: 37.6x
  행렬곱 - NumPy: 0.0003s

📏 크기: 1000
  덧셈 - 벡터: 0.0006s, 스칼라: 0.0936s, 속도향상: 148.8x
  행렬곱 - NumPy: 0.0132s

📏 크기: 10000
  덧셈 - 벡터: 0.0022s, 스칼라: 0.9656s, 속도향상: 446.7x


## 10. Batch Normalization 실습

In [47]:
# Batch Normalization 효과 확인
print("🔄 Batch Normalization 테스트")
print("=" * 40)

# 입력 데이터 (평균과 분산이 다양함)
X = np.random.randn(32, 10) * 5 + 3  # 평균 3, 표준편차 5

print(f"원본 데이터:")
print(f"  평균: {X.mean():.2f}")
print(f"  표준편차: {X.std():.2f}")

# Batch Norm 적용
gamma = np.ones(10)
beta = np.zeros(10)

X_norm, (mean, var) = batch_norm(X, gamma, beta, training=True)

print(f"\n정규화 후:")
print(f"  평균: {X_norm.mean():.4f}")
print(f"  표준편차: {X_norm.std():.4f}")

# 각 특성별 통계
print(f"\n특성별 통계 (처음 5개):")
for i in range(min(5, X.shape[1])):
    print(f"  특성 {i}: 평균={X_norm[:, i].mean():.4f}, 분산={X_norm[:, i].var():.4f}")

🔄 Batch Normalization 테스트
원본 데이터:
  평균: 3.26
  표준편차: 5.09

정규화 후:
  평균: 0.0000
  표준편차: 1.0000

특성별 통계 (처음 5개):
  특성 0: 평균=0.0000, 분산=1.0000
  특성 1: 평균=0.0000, 분산=1.0000
  특성 2: 평균=0.0000, 분산=1.0000
  특성 3: 평균=0.0000, 분산=1.0000
  특성 4: 평균=-0.0000, 분산=1.0000


## 11. 손실 함수 비교

In [48]:
# 다양한 손실 함수 비교
print("📉 손실 함수 비교")
print("=" * 40)

# 회귀용 데이터
y_true_reg = np.array([1.0, 2.0, 3.0, 4.0])
y_pred_reg = np.array([1.2, 2.3, 2.8, 4.1])

mse = mse_loss(y_pred_reg, y_true_reg)
print(f"\n회귀 손실:")
print(f"  MSE Loss: {mse:.4f}")

# 분류용 데이터
y_true_cls = np.array([0, 1, 2, 1])
y_pred_cls = np.array([[0.8, 0.1, 0.1],
                       [0.1, 0.7, 0.2],
                       [0.2, 0.2, 0.6],
                       [0.1, 0.8, 0.1]])

ce = cross_entropy(y_pred_cls, y_true_cls)
acc = accuracy(y_pred_cls, y_true_cls)

print(f"\n분류 손실:")
print(f"  Cross Entropy: {ce:.4f}")
print(f"  Accuracy: {acc:.2%}")

# 완벽한 예측 vs 랜덤 예측
print(f"\n극단적인 경우:")

# 완벽한 예측
y_perfect = np.array([[1, 0, 0],
                      [0, 1, 0],
                      [0, 0, 1]])
y_true_perfect = np.array([0, 1, 2])
ce_perfect = cross_entropy(y_perfect, y_true_perfect)
print(f"  완벽한 예측 CE: {ce_perfect:.6f}")

# 균등 분포 (랜덤)
y_random = np.ones((3, 3)) / 3
ce_random = cross_entropy(y_random, y_true_perfect)
print(f"  랜덤 예측 CE: {ce_random:.4f}")

📉 손실 함수 비교

회귀 손실:
  MSE Loss: 0.0450

분류 손실:
  Cross Entropy: 0.3284
  Accuracy: 100.00%

극단적인 경우:
  완벽한 예측 CE: 0.000000
  랜덤 예측 CE: 1.0986


## 12. 메모리 효율성 분석

In [29]:
# Broadcasting vs Explicit 복사
print("💾 메모리 효율성 비교")
print("=" * 40)

size = 10000
a = np.random.randn(size, 100)
b = np.random.randn(100)

# Broadcasting (효율적)
start = time.time()
result1 = a + b  # b가 자동으로 broadcast
broadcast_time = time.time() - start

# Explicit 복사 (비효율적)
start = time.time()
b_repeated = np.tile(b, (size, 1))
result2 = a + b_repeated
explicit_time = time.time() - start

print(f"배열 크기: {a.shape}")
print(f"\nBroadcasting:")
print(f"  시간: {broadcast_time:.4f}초")
print(f"  메모리: b는 (100,) 유지")

print(f"\nExplicit 복사:")
print(f"  시간: {explicit_time:.4f}초")
print(f"  메모리: b_repeated는 {b_repeated.shape}")

print(f"\n효율성: {explicit_time/broadcast_time:.1f}배 빠름!")

# 결과 동일성 확인
print(f"\n결과 동일: {np.allclose(result1, result2)}")

💾 메모리 효율성 비교
배열 크기: (10000, 100)

Broadcasting:
  시간: 0.0011초
  메모리: b는 (100,) 유지

Explicit 복사:
  시간: 0.0023초
  메모리: b_repeated는 (10000, 100)

효율성: 2.2배 빠름!

결과 동일: True


## 🎉 축하합니다!

벡터화된 연산의 힘을 경험하셨습니다!

### 핵심 교훈:
1. **벡터화는 필수**: 10배 이상의 속도 향상
2. **Broadcasting 활용**: 메모리 효율적인 연산
3. **Batch 처리**: 병렬 연산으로 처리량 증가
4. **수치 안정성**: 오버플로우/언더플로우 방지

### 다음 단계:
- `50_eval/mnist_mini.py` 실행하여 실제 데이터셋 학습
- `50_eval/benchmark.py` 실행하여 성능 비교
- Day 3: Attention Mechanism으로 진행